In [49]:
from aicspylibczi import CziFile
import numpy as np
import skimage as sk
import pandas as pd
import os
from glob import glob
import napari
from cellpose import models, io, utils
#import pyclesperanto as cle


In [ ]:
def normalize_images(input_image,tophat_radius):
    input_gpu = cle.push(input_image)
    #normalizing the image stack
    equalized_intensities_stack = cle.create_like(input_gpu)
    a_slice = cle.create([input_gpu.shape[-2], input_gpu.shape[-1]])
    num_slices = input_gpu.shape[1]
    num_channels = input_gpu.shape[0]
    if num_channels == 1:
        mean_intensity_stack = cle.mean_of_all_pixels(input_gpu)
        corrected_slice = None
        for z in range(0, num_slices):
            # get a single slice out of the stack
            cle.copy_slice(input_gpu, a_slice, z)
            # measure its intensity
            mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
            # correct the intensity
            correction_factor = mean_intensity_slice/mean_intensity_stack
            corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
            # copy slice back in a stack
            cle.copy_slice(corrected_slice, equalized_intensities_stack, z)
    else:
        for channel in range(num_channels):
            mean_intensity_stack = cle.mean_of_all_pixels(input_gpu[channel,...])
            corrected_slice = None
            for z in range(0, num_slices):
                # get a single slice out of the stack
                cle.copy_slice(input_gpu[channel,...], a_slice, z)
                # measure its intensity
                mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
                # correct the intensity
                correction_factor = mean_intensity_slice/mean_intensity_stack
                corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
                # copy slice back in a stack
                cle.copy_slice(corrected_slice, equalized_intensities_stack[channel], z)
    #background subtraction (increase the signal to noise ratio for improved segmentation results)
    background_subtracted_top_hat = cle.top_hat_sphere(equalized_intensities_stack,radius_x=tophat_radius,radius_y=tophat_radius,radius_z=tophat_radius)
    #pull data off gpu
    input_pull = cle.pull(input_gpu)
    background_subtracted_top_hat_pull = cle.pull(background_subtracted_top_hat)
    equalized_intensities_stack_pull = cle.pull(equalized_intensities_stack)
    return background_subtracted_top_hat_pull

In [38]:
def normalize_images_CPU(input):
    equalized_stack = np.zeros_like(input)
    num_slices = input.shape[1]
    num_channels = input.shape[0]
    if num_channels > 1:
        for channel in range(num_channels):
            channel_stack = input[channel,...]
            mean_intensity_stack = sk.filters.threshold_mean(channel_stack)
            corrected_stack = np.astype(np.stack([channel_stack[z,...]*(sk.filters.threshold_mean(channel_stack[z,...])/mean_intensity_stack) for z in range(num_slices)], axis=0),np.uint16)
            background_subtracted = np.astype(sk.morphology.white_tophat(corrected_stack),np.uint16)
            np.copyto(equalized_stack[channel,...],background_subtracted)
    else:
        mean_intensity_stack = sk.filters.threshold_mean(input)
        corrected_stack = np.astype(np.stack([input[z,...]*(sk.filters.threshold_mean(input[z,...])/mean_intensity_stack) for z in range(num_slices)], axis=0),np.uint16)
        background_subtracted = sk.morphology.white_tophat(corrected_stack)
        np.copyto(equalized_stack,background_subtracted)
    return equalized_stack           


In [ ]:
#cle.select_device("NVIDIA")

(OpenCL) NVIDIA RTX A4000 (OpenCL 3.0 CUDA)
	Vendor:                      NVIDIA Corporation
	Driver Version:              573.24
	Device Type:                 GPU
	Compute Units:               48
	Global Memory Size:          16375 MB
	Local Memory Size:           0 MB
	Maximum Buffer Size:         4093 MB
	Max Clock Frequency:         1560 MHz
	Image Support:               Yes

In [3]:
file_paths = sorted(glob("E:\Fondufe-Mittendorf_Lab\OIC-150_Images\*.czi"))
czi_files = list(map(CziFile,file_paths))

In [4]:
test_img = czi_files[0]

In [45]:
print(test_img.meta)

<Element 'ImageDocument' at 0x0000016C81BF6890>


In [13]:
test_img.size

(1, 1, 3, 1, 4, 24, 1012, 1012)

In [43]:
help(CziFile)

Help on class CziFile in module aicspylibczi.CziFile:

class CziFile(builtins.object)
 |  CziFile(czi_filename: Union[str, pathlib.Path, bytes, BinaryIO, io.BufferedIOBase, _io.BufferedReader], verbose: bool = False)
 |  
 |  Zeiss CZI file object.
 |  
 |  Args:
 |    |  czi_filename (str): Filename of czifile to access.
 |  
 |  Kwargs:
 |    |  verbose (bool): Print information and times during czi file access.
 |  
 |  .. note::
 |  
 |     Utilizes compiled wrapper to libCZI for accessing the CZI file.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, czi_filename: Union[str, pathlib.Path, bytes, BinaryIO, io.BufferedIOBase, _io.BufferedReader], verbose: bool = False)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  get_all_mosaic_scene_bounding_boxes(self)
 |      Get the scene (subblocks) bounding boxes (pyramid=0) for the specified dimensions.
 |      For mosaic files.
 |      
 |      Returns
 |      -------
 |      dict[int, bbox]
 |       

In [5]:
img = [test_img.read_image(S=i)[0] for i in range(test_img.size[2])]

In [6]:
img = np.squeeze(img)

In [10]:
test = img[0]
test.shape[0]

4

In [14]:
footprint = sk.morphology.disk(2)
equalized_stack = np.zeros_like(test)
num_slices = test.shape[1]

In [30]:

channel_stack = test[0,...]
mean_intensity_stack = sk.filters.threshold_mean(channel_stack)
corrected_stack = np.astype(np.stack([channel_stack[z,...]*(sk.filters.threshold_mean(channel_stack[z,...])/mean_intensity_stack) for z in range(num_slices)],axis=0),np.uint16)
background_subtracted_01 = np.astype(sk.morphology.white_tophat(corrected_stack),np.uint16)


In [33]:
channel_stack = test[1,...]
mean_intensity_stack = sk.filters.threshold_mean(channel_stack)
corrected_stack = np.astype(np.stack([channel_stack[z,...]*(sk.filters.threshold_mean(channel_stack[z,...])/mean_intensity_stack) for z in range(num_slices)],axis=0),np.uint16)
background_subtracted_02 = np.astype(sk.morphology.white_tophat(corrected_stack),np.uint16)

In [31]:
corrected_stack.shape

(24, 1012, 1012)

In [34]:
merged = np.stack((background_subtracted_01, background_subtracted_02), axis = 0)

In [35]:
merged.shape

(2, 24, 1012, 1012)

In [39]:
norm_img= normalize_images_CPU(test)

In [48]:
viewer = napari.view_image(test[3], name="input", scale=(0.25,0.13,0.13))